# POC Model

__Install packages__

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords
import glob
from datetime import datetime
import statistics 
import pickle
import jmespath
import json
from rake_nltk import Rake
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from gensim.test.utils import datapath
import math
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
from collections import Counter
%load_ext autotime

C:\ProgramData\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\ProgramData\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Converting Tree to JSON format

In [2]:
# Please run this code cell once only as multiple runs will affect the same file 
with open('Clean_mind_map.json') as json_file:
    data = json.load(json_file)
    
itemsKeyedById = {i["ID"]: i for i in data}

#iterate through each item in the `myJson` list.
for item in data[1:]:
    #does the item have a parent?
    if "Parent" in item:
        #get the parent item
        parent = itemsKeyedById[item['Parent']]
        #if the parent item doesn't have a "children" member, 
        #we must create one.
        if "children" not in parent:
            parent["children"] = []
        #add the item to its parent's "children" list.
        parent["children"].append(item)
topLevelItems = [item for item in data if "Parent" not in item]
taxonomy_data = {'data': data}

In [3]:
# with open('data.json', 'w') as outfile:
#     json.dump(taxonomy_data, outfile)

Examples are found on this link: http://jmespath.org/tutorial.html

In [4]:
expression = jmespath.compile("data[?Title =='Lean of Peak'].children[].Title")
expression.search(data)

### Data Sources Integration

In [162]:
data = []
for filename in glob.glob('*.csv'):
    data.append(filename)
data

['COPA_blog.csv',
 'COPA_forum_replies.csv',
 'COPA_forum_threads.csv',
 'COPA_Wiki.csv']

In [163]:
# magazine = pd.read_csv(data[])
# magazine_date_list = list(magazine['Magazine'].unique())
# # Get the month of magazine from its name
# # Since the the COPA magazines are released monthly, so it only has year and month
# # in its date column. In this case we just assume all magazines were released on the 
# # first day of that month.
# magazine_date_dict = {magazine_date_list[0]: '2015-01-01', magazine_date_list[1]: '2019-01-01',
#                       magazine_date_list[2]: '2015-07-01', magazine_date_list[3]: '2019-04-01',
#                       magazine_date_list[4]: '2019-06-01', magazine_date_list[5]: '2018-01-01',
#                       magazine_date_list[6]: '2017-09-01', magazine_date_list[7]: '2016-09-01',
#                       magazine_date_list[8]: '2017-11-01', magazine_date_list[9]: '2015-05-01',
#                       magazine_date_list[10]: '2015-09-01', magazine_date_list[11]: '2006-09-01', 
#                       magazine_date_list[12]: '2018-06-01', magazine_date_list[13]: '2019-07-01',
#                       magazine_date_list[14]: '2017-04-01', magazine_date_list[15]: '2016-04-01',
#                       magazine_date_list[16]: '2018-11-01', magazine_date_list[17]: '2015-09-01',
#                       magazine_date_list[18]: '2019-03-01', magazine_date_list[19]: '2006-11-01',
#                       magazine_date_list[20]: '2016-01-01', magazine_date_list[21]: '2019-05-01',
#                       magazine_date_list[22]: '2016-03-01', magazine_date_list[23]: '2016-05-01',
#                       magazine_date_list[24]: '2006-07-01', magazine_date_list[25]: '2017-06-01',
#                       magazine_date_list[26]: '2018-07-01', magazine_date_list[27]: '2018-09-01',
#                       magazine_date_list[28]: '2017-01-01', magazine_date_list[29]: '2012-11-01'}
# magazine['Magazine'] = magazine['Magazine'].map(magazine_date_dict)
# magazine.columns = ['Date', 'Title', 'Content', 'Author']
# magazine['Like'] = 0
# magazine['Comment'] = 0
# magazine['Resource'] = 'Magazine'

In [164]:
wiki = pd.read_csv(data[3])
wiki = wiki[['LastModifiedUtcDate','UserId', 'Title', 'FormattedBody', 'TotalViews']]
wiki['LastModifiedUtcDate'] = wiki['LastModifiedUtcDate'].apply(lambda x: x[0:10])
wiki.columns = ['Date', 'UserID', 'Title', 'FormattedBody', 'TotalViews']
wiki['Source'] = 'Wiki'
wiki.head()

,Date,UserID,Title,FormattedBody,TotalViews,Source
0,2008-12-21,2256,Cirrus accident discussions,asdf\r\n\r\n\r\n\r\nAccident\r\nDate\r\nLocati...,361,Wiki
1,2008-12-21,2256,"Cirrus Fatal Accident #01 near Sierra Vista, AZ",\r\n\r\n\r\nCirrus Fatal Accident #1Sierra Vis...,169,Wiki
2,2008-12-21,2256,First page,http://www.cirruspilots.org/wikis/sandbox/add....,294,Wiki
3,2008-12-21,2256,Test infobox for accidents,\r\n\r\n\r\nCirrus Fatal Accident #40Rock Spri...,57,Wiki
4,2008-12-21,2256,"Cirrus Fatal Accident #33 near Paso Robles, CA",\r\n\r\n\r\nCirrus Fatal Accident #33Paso Robl...,318,Wiki


In [165]:
blog = pd.read_csv(data[0])
blog = blog[['PostDate', 'UserId', 'PostName', 'PostBody', 'TotalViews']]
blog.columns = ['Date', 'UserID', 'Title', 'FormattedBody', 'TotalViews']
blog['Source'] = 'Blog'
blog.head()

,Date,UserID,Title,FormattedBody,TotalViews,Source
0,2008-09-16,2438,holds,\r\nDuring advanced instruction and in particu...,1345,Blog
1,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856,Blog
2,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856,Blog
3,2010-10-19,2438,do you know how to do this a top 5 list of ite...,\r\nAs an ongoing series of blog posts revolvi...,1747,Blog
4,2014-02-12,4736,copa sun and fun lunch april 4 2014 lake land fl,"Rich Hayes, Governor of COPA South East invite...",285,Blog


In [166]:
# Load forum replies
forum = pd.read_csv(data[1])
forum = forum[['ThreadReplyDate', 'UserID', 'FormattedBody', 'ThreadId', 'UserPoints']]
forum.columns = ['Date', 'UserID', 'FormattedBody', 'ThreadId', 'UserPoints']
forum['Date'] = forum['Date'].apply(lambda x: x.split(' ')[0])
forum['Source'] = 'Forum'
forum.head()

,Date,UserID,FormattedBody,ThreadId,UserPoints,Source
0,2008-06-25,3870,You are cordially invited to a Cirrus BBQ & Fu...,28054,32,Forum
1,2005-10-23,4255,"Ed,Thank you for the offer. My envelope is on ...",16,239,Forum
2,2005-10-23,3426,If I come over Tuesday do I get to play Igor t...,16,35,Forum
3,2005-10-24,2296,"Ed, My envelope is on the way also! thanks for...",16,17,Forum
4,2005-11-04,3586,"When I returned home from S FL yesterday, I co...",16,182,Forum


In [167]:
df = pd.concat([blog, forum], ignore_index=True, sort=False)
df = pd.concat([df, wiki], ignore_index=True, sort=False)
df.head()

,Date,UserID,Title,FormattedBody,TotalViews,Source,ThreadId,UserPoints
0,2008-09-16,2438,holds,\r\nDuring advanced instruction and in particu...,1345.0,Blog,NaN,NaN
1,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN
2,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN
3,2010-10-19,2438,do you know how to do this a top 5 list of ite...,\r\nAs an ongoing series of blog posts revolvi...,1747.0,Blog,NaN,NaN
4,2014-02-12,4736,copa sun and fun lunch april 4 2014 lake land fl,"Rich Hayes, Governor of COPA South East invite...",285.0,Blog,NaN,NaN


In [168]:
#Ranking the source, blog is the most important source so have 3
source_dict = {'Blog': 3, 'Forum': 2, 'Wiki': 1}
df['SourceScore'] = df['Source'].map(source_dict)
df.head()

,Date,UserID,Title,FormattedBody,TotalViews,Source,ThreadId,UserPoints,SourceScore
0,2008-09-16,2438,holds,\r\nDuring advanced instruction and in particu...,1345.0,Blog,NaN,NaN,3
1,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN,3
2,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN,3
3,2010-10-19,2438,do you know how to do this a top 5 list of ite...,\r\nAs an ongoing series of blog posts revolvi...,1747.0,Blog,NaN,NaN,3
4,2014-02-12,4736,copa sun and fun lunch april 4 2014 lake land fl,"Rich Hayes, Governor of COPA South East invite...",285.0,Blog,NaN,NaN,3


In [169]:
df[['FormattedBody']] = df[['FormattedBody']].fillna(value='No text.')

# Generate Attributes

__Generate Recency Score__

In [170]:
def  Calculate_RecencyScore(date):
    '''
    Recency Rate = log( 1 + 1/(days between the post date and current date + 1))
    '''
    date_datetime = datetime.strptime(date, '%Y-%m-%d').date()
    rececncy_rate = math.log10(1/(((datetime.date(datetime.now()))-date_datetime).days+1)+1)
    return rececncy_rate
    

df['RecencyRate'] = df['Date'].map(lambda y: Calculate_RecencyScore(y))

__Generate Author Score__

In [171]:
author_list = list(df['UserID'].unique())
author_score_dict = {}
for author in author_list:
    author_score = df[df['UserID'] == author]['SourceScore'].sum()
    author_score_dict[author] = author_score
    
df['AuthorScore'] = df['UserID'].map(author_score_dict)

__Generate pre-Content Score__

In [286]:
#Calculate pre-content score without taking the search query into consideration

In [182]:
# This function will assign each row in a dataframe a topic according to the pre-trained topic model.
def Assign_Topic(df):
    
    # Load pre-trained model
    filename = 'lda_model.pickle'
    infile = open(filename,'rb')
    lda_model = pickle.load(infile)

    filename = 'id2word.pickle'
    infile = open(filename,'rb')
    id2word = pickle.load(infile)

    stop_words = stopwords.words('english')

    data = df.FormattedBody.values.tolist()
    data_cleanlist = []
    for d in data:
        try:
            data_cleanlist.append(d.lower())
        except:
            continue
            
    # Put contents into model and assign each content to a topic
    texts1 = [[word for word in doc.split() if word not in stop_words] for doc in data_cleanlist]
    corpus1 = [id2word.doc2bow(text) for text in texts1]
    resultlist = lda_model.get_document_topics(corpus1)
    
    # Build a list containing the topic classification
    topic_lst = []
    for i, result in enumerate(resultlist):
        temp = 0
        topic = 0
        for j in range(len(result)):
            if result[j][1] > temp:
                temp = result[j][1]
                topic = j
        topic_lst.append(topic)
        
    df['TopicID'] = topic_lst
    
    return df

In [285]:
df_withTopic = Assign_Topic(df)

In [226]:
# Generate a dataframe that store the topic id and its keywords-weight dictionary
lda_topics = lda_model.print_topics(num_words = 10)
topic_dict_lst = []
topic_keywords_lst = []

for topic in lda_topics:
    keywords = re.findall(r'"(.+?)"',topic[1])
    topic_keywords_lst.append(keywords)
    
    weights = re.findall(r'([0]\..+?)\*',topic[1])
    
    topic_dict = dict(zip(keywords, weights))
    topic_dict_lst.append(topic_dict)

In [291]:
# Generate body_words_lst which contains sublists that hold all words in a formattedBody
data = df_withTopic.FormattedBody.values.tolist()
df_cleanlist = []
for d in data:
    try:
        df_cleanlist.append(d.lower())
    except:
        continue

body_words_lst = [[word for word in doc.split() if word not in stop_words] for doc in df_cleanlist]

time: 2min 13s


In [293]:
# Generate the list of pre-content score
topic_id_lst = list(df_withTopic.TopicID)
pre_content_score_lst = list()

for row_index in range(len(df_withTopic)):
    
    topic_id = topic_id_lst[row_index]
    
    keyword_vector = topic_keywords_lst[topic_id]
    body_words = body_words_lst[row_index]

    words_hit = [word for word in body_words if word in keyword_vector]
    frequency_count = Counter(words_hit)
    
    pre_content_score = 0
    for hitted_word in frequency_count.keys():
        weight = float(topic_dict_lst[topic_id][hitted_word])
        count = frequency_count[hitted_word]
        pre_content_score += count*weight
    
    pre_content_score_lst.append(pre_content_score)

time: 11 s


In [331]:
df_withTopic['PreContentScore'] = pre_content_score_lst
df = df_withTopic.copy()

time: 363 ms


__Generate Content Score__

In [11]:
# Load pre-trained model
filename = 'lda_model.pickle'
infile = open(filename,'rb')
lda_model = pickle.load(infile)

filename = 'id2word.pickle'
infile = open(filename,'rb')
id2word = pickle.load(infile)

stop_words = stopwords.words('english')

time: 260 ms


In [12]:
def topic_match(search_query):
    '''
    Extract the topic of searching query by LDA model trained by forum posts.
    '''
    texts1 = [[word for word in doc.lower().split() if word not in stop_words] for doc in [search_query]]
    corpus1 = [id2word.doc2bow(t) for t in texts1]
    result = lda_model.get_document_topics(corpus1)
    LDA_topic = max(result[0], key=lambda x: x[1])[0]
    return LDA_topic

time: 1.05 ms


In [14]:
def Calculate_ContentScore(search_query):
    
    query_topic_id = topic_match(search_query)
    df['ContentScore'] = df['PreContentScore']
    
    # Expand parameter here I choose 5.
    df.loc[(df.TopicID == query_topic_id),'ContentScore']= df.loc[(df.TopicID == query_topic_id),'PreContentScore']*5
    
    return df

time: 999 µs


In [15]:
df_withALLScore = Calculate_ContentScore('Cirrus Parachute First Responders')

time: 58 ms


__Calculate All Score__

In [16]:
df_withALLScore

,Unnamed: 0,Date,UserID,Title,FormattedBody,TotalViews,Source,ThreadId,UserPoints,SourceScore,RecencyRate,AuthorScore,TopicID,PreContentScore,ContentScore
0,0,2008-09-16,2438,holds,\r\nDuring advanced instruction and in particu...,1345.0,Blog,NaN,NaN,3,0.000104,2504,8,0.000,0.000
1,1,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN,3,0.000129,359,2,0.000,0.000
2,2,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN,3,0.000129,359,3,0.018,0.018
3,3,2010-10-19,2438,do you know how to do this a top 5 list of ite...,\r\nAs an ongoing series of blog posts revolvi...,1747.0,Blog,NaN,NaN,3,0.000127,2504,5,0.000,0.000
4,4,2014-02-12,4736,copa sun and fun lunch april 4 2014 lake land fl,"Rich Hayes, Governor of COPA South East invite...",285.0,Blog,NaN,NaN,3,0.000196,4851,8,0.038,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038897,1038897,2017-11-26,14058,United Kingdom and Ireland - Wiki,No text.,85.0,Wiki,NaN,NaN,1,0.000525,873,0,0.000,0.000
1038898,1038898,2017-11-30,14058,United Kingdom and Ireland Regional Leadership...,No text.,0.0,Wiki,NaN,NaN,1,0.000527,873,0,0.000,0.000
1038899,1038899,2018-09-19,19647,NexAir Avionics - Wiki,No text.,3.0,Wiki,NaN,NaN,1,0.000819,2,0,0.000,0.000
1038900,1038900,2019-01-03,19647,Finance Consult - Wiki,No text.,0.0,Wiki,NaN,NaN,1,0.001023,2,0,0.000,0.000


time: 23 ms


__Normalization__

In [17]:
def Normalization(col):
    nomolized_col = (df_withALLScore[col] - df_withALLScore[col].min()) / (df_withALLScore[col].max() - df_withALLScore[col].min())
    return nomolized_col

df_withALLScore['RecencyRate'] = Normalization('RecencyRate')
df_withALLScore['AuthorScore'] = Normalization('AuthorScore')
df_withALLScore['ContentScore'] = Normalization('ContentScore')

time: 47.3 ms


In [18]:
df_withALLScore

,Unnamed: 0,Date,UserID,Title,FormattedBody,TotalViews,Source,ThreadId,UserPoints,SourceScore,RecencyRate,AuthorScore,TopicID,PreContentScore,ContentScore
0,0,2008-09-16,2438,holds,\r\nDuring advanced instruction and in particu...,1345.0,Blog,NaN,NaN,3,0.016162,0.045504,8,0.000,0.000000
1,1,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN,3,0.025377,0.006508,2,0.000,0.000000
2,2,2010-12-26,7912,sunday night in anguilla,"Well, I survived Christmas Eve and Christmas. ...",856.0,Blog,NaN,NaN,3,0.025377,0.006508,3,0.018,0.001248
3,3,2010-10-19,2438,do you know how to do this a top 5 list of ite...,\r\nAs an ongoing series of blog posts revolvi...,1747.0,Blog,NaN,NaN,3,0.024455,0.045504,5,0.000,0.000000
4,4,2014-02-12,4736,copa sun and fun lunch april 4 2014 lake land fl,"Rich Hayes, Governor of COPA South East invite...",285.0,Blog,NaN,NaN,3,0.049400,0.088172,8,0.038,0.002635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038897,1038897,2017-11-26,14058,United Kingdom and Ireland - Wiki,No text.,85.0,Wiki,NaN,NaN,1,0.167127,0.015853,0,0.000,0.000000
1038898,1038898,2017-11-30,14058,United Kingdom and Ireland Regional Leadership...,No text.,0.0,Wiki,NaN,NaN,1,0.168041,0.015853,0,0.000,0.000000
1038899,1038899,2018-09-19,19647,NexAir Avionics - Wiki,No text.,3.0,Wiki,NaN,NaN,1,0.272472,0.000018,0,0.000,0.000000
1038900,1038900,2019-01-03,19647,Finance Consult - Wiki,No text.,0.0,Wiki,NaN,NaN,1,0.345763,0.000018,0,0.000,0.000000


time: 19 ms


In [38]:
# First weight combination1
df_withALLScore['TotalScore1'] = df_withALLScore['RecencyRate']*(1/10) + df_withALLScore['AuthorScore']*(4/10)  + df_withALLScore['ContentScore']*(5/10)
combination1 = df_withALLScore.sort_values("TotalScore1", inplace = False,ascending=False).head(5)
for index in range(5):
    print("------------------------------------------The result %d-----------------------------------------"%(index+1))
    print(list(combination1.FormattedBody)[index])

------------------------------------------The result 1-----------------------------------------
NTSB Factual report released.  I did not know this was his first flight after his training to pilot the MU-2.  I don't understand the NTSB's comments about Flight Data Recorder and making a big deal that the FAA is not implementing their guidance to force installation on Turbine aircraft.  This airplane had a Garmin G600.  It records all aspects of the airplane attitude.  This report makes no mention of the NTSB trying to recover that information.  Huh?
CEN14FA046


HISTORY OF FLIGHTOn November 10, 2013, about 1546 central standard time, a Mitsubishi MU-2B-25 twin-engine airplane, N856JT, impacted wooded terrain while maneuvering near Owasso, Oklahoma. The commercial pilot, who was the sole occupant of the airplane, sustained fatal injuries. The airplane was destroyed. The airplane was registered to Anasazi Winds, LLC, Tulsa, Oklahoma, and was operated by the pilot under the provisions of 14 Code of Federal Regulations Part 91 as a personal flight. Visual meteorological conditions prevailed for the flight, and an instrument flight plan had been filed. The flight departed Salina Regional Airport (SLN), Salina, Kansas, about 1503 and was en route to Tulsa International Airport (TUL), Tulsa, Oklahoma.After takeoff, the airplane was radar identified by the Kansas City Center (ZKC) sector R66 controller, and the pilot was cleared to climb to 9,000 feet. About 1506, the pilot was cleared to climb to 17,000 feet. The flight proceeded normally, and at 151

------------------------------------------The result 2-----------------------------------------
NTSB has released their factual report on this accident (appended below). Also, lots of material in the public information docket.
 

NTSB Identification: ERA11FA41414 CFR Part 91: General AviationAccident occurred Saturday, July 23, 2011 in Palm Bay, FLAircraft: CIRRUS DESIGN CORP SR22, registration: N122HBInjuries: 2 Fatal,1 Minor.


 
NTSB investigators either traveled in support of this investigation or conducted a significant amount of investigative work without any travel, and used data obtained from various sources to prepare this aircraft accident report.
 
HISTORY OF FLIGHT


On July 23, 2011, about 1233 eastern daylight time, a Cirrus Design Corporation SR22, N122HB, registered to N122HB LLC, and operated by a private individual, experienced a loss of engine power and sustained substantial damage during an attempted forced landing to an open field near Palm Bay, Florida. Visual meteorological conditions prevailed at the time and an instrument flight rules (IFR) flight plan was filed for the 14 Code of Federal Regulations (CFR) Part 91 personal flight from Albert Whitted Airport (SPG), St. Petersburg, Florida, to Melbourne International Airport (MLB), Melbourne, Florida. The private pilot and one passenger were fatally injured, and one passenger sustained minor injuries. The flight originated from SPG about 1145.The rear seat passenger stated that before takeoff, the pilot, “…checked the airplane on the outside as well as all the instruments inside the airplane.” She reported the takeoff was perfect and was very smooth and she felt very safe.According to a 

------------------------------------------The result 3-----------------------------------------
[quote user="Brian Rose"]Did you do any short(er) 3500ft ish runways?  Their test/demo pilot refused to even attempt for us :([/quote]No, Santa Barbara, 6000 feet long.
------------------------------------------The result 4-----------------------------------------
Scott, where do I begin? I can only sit here so long on the sidelines are listen to you whine defensively and spout the same old BS you always prattle on about. You personify the reason I never considered a (then) Lancair aircraft when I was in the market. Arrogance is not a good quality in sales or management, as you and your mark have so aptly demonstrated to the GA community. You say the 400 is a superior aircraft to the G3 Turbo but I just don't see it. On what basis?  Please don't trot out the tired old refrain about Utility category. Please. Are you inferring that an aircraft certified under the new Part 23 in the Normal cate

In [39]:
# First weight combination2
df_withALLScore['TotalScore2'] = df_withALLScore['RecencyRate']*(3/10) + df_withALLScore['AuthorScore']*(3/10)  + df_withALLScore['ContentScore']*(4/10)
combination2 =df_withALLScore.sort_values("TotalScore2", inplace = False,ascending=False).head(5)
for index in range(5):
    print("------------------------------------------The result %d-----------------------------------------"%(index+1))
    print(list(combination2.FormattedBody)[index])

------------------------------------------The result 1-----------------------------------------
[quote user="Brian Rose"]Did you do any short(er) 3500ft ish runways?  Their test/demo pilot refused to even attempt for us :([/quote]No, Santa Barbara, 6000 feet long.
------------------------------------------The result 2-----------------------------------------
We all know Dennis using that to help his "stick and rudder"!
------------------------------------------The result 3-----------------------------------------
[quote user="Jim Grace"]I took a few photos of my setup and I'll walk through each one.[/quote]This is a great post and will be a go-to referral for future questions on how to do this. Thanks for making it!
[quote user="Jim Grace"]At the supply port, I purchased a 90 degree metal CPC connector from Mountain High. [/quote]The only comment I have here is that I would not get the metal one but rather the plastic one. It probably doesn't make much practical difference, it's just t

In [40]:
# First weight combination3
df_withALLScore['TotalScore3'] = df_withALLScore['RecencyRate']*(1/3) + df_withALLScore['AuthorScore']*(1/3)  + df_withALLScore['ContentScore']*(1/3)
combination3 =df_withALLScore.sort_values("TotalScore3", inplace = False,ascending=False).head(5)
for index in range(5):
    print("------------------------------------------The result %d-----------------------------------------"%(index+1))
    print(list(combination3.FormattedBody)[index])

------------------------------------------The result 1-----------------------------------------
[quote user="Jim Grace"]I took a few photos of my setup and I'll walk through each one.[/quote]This is a great post and will be a go-to referral for future questions on how to do this. Thanks for making it!
[quote user="Jim Grace"]At the supply port, I purchased a 90 degree metal CPC connector from Mountain High. [/quote]The only comment I have here is that I would not get the metal one but rather the plastic one. It probably doesn't make much practical difference, it's just that I didn't like a square metal thing sticking into the cabin in the unlikely case of an accident where a stray head bouncing around the cabin could come in contact with it. 
------------------------------------------The result 2-----------------------------------------
We all know Dennis using that to help his "stick and rudder"!
------------------------------------------The result 3-----------------------------------

# Compare keyword vector with reference vector

__Match replies with searching terms by topic only__

In [297]:
def topic_match(search_query):
    '''
    Extract the topic of searching query by LDA model trained by forum posts.
    '''
    texts1 = [[word for word in doc.lower().split() if word not in stop_words] for doc in [search_query]]
    corpus1 = [id2word.doc2bow(t) for t in texts1]
    result = lda_model.get_document_topics(corpus1)
    LDA_topic = max(result[0], key=lambda x: x[1])[0]
    return LDA_topic

time: 1.98 ms


In [298]:
reference_vector = topic_match('Cirrus Parachute First Responders')
matched_index = [key for key, value in topic_dict.items() if value == reference_vector]
reply_match_search = forum.iloc[matched_index]

time: 22 ms


In [55]:
reference_vector, lda_model.print_topics(num_words = 10)[reference_vector][1]

(13,
 '0.100*"pilot" + 0.034*"fly" + 0.030*"training" + 0.024*"accident" + 0.022*"cap" + 0.021*"airplane" + 0.020*"cirrus" + 0.017*"flight" + 0.017*"plane" + 0.014*"safety"')

In [60]:
result = reply_match_search['FormattedBody'].reset_index(drop = True)
for i in range(len(result)):
    print (result[i] + '\n')

In reply to:Even if CO doesn't turn out to be the cause I am beating myself up mercilessly over NOT having CO detection in my cockpit. What is the best one out there?I have from Sporty's the silver colored industrial electronic CO2 detector that sells for $295. (They have several listed in their catalog).  When I took delivery of my SR 20 three years ago, serial # 1604, while taking the factory transition training the CO2 detector went off when it was in a flight bag in the luggage compartment.  It turned out that the exhaust manifold was damaged.  There was no odor in the cockpit - the CO2 detector triggered during a long cross country flight.  The factory was surprised the detector went off.  The factory ended up replacing the exhaust manifold and there has been no problem since.  I currently put the C02 detector in the little "glove box" hole above the heating controls of my SR 20 where I also keep the checklist, a pad, and pencils.SR20 Cirrus N356JP, Serial # 1604 

Same battery yo

__Match replies with searching terms by comparing keywords vector and reference vector__

In [331]:
r = re.compile(r'([.0-9]+)\*\"([a-zA-Z]+)\"')
keyword_vector = {}
for i in range(20):
    keyword_vector[i] = {}
    word_weight = r.findall(lda_model.print_topics(num_words = 10)[i][1])
    for weight, word in word_weight:
        keyword_vector[i][word] = weight

In [332]:
# Compare query to each leaf node, if the partial match score is higher than the threshold, then return a combination vector
# that contains all words in those leaf nodes.
def fuzzywuzzy_match(query,threshold):
    
    # Generate a list contains all leaf nodes, each sublist contains all words in that leaf node.
    leaf_node = list()
    for item in topic_vector:
        if len(topic_vector[item]) == 1:
            leaf_node.append(topic_vector[item][0].split())
            
    reference_vector = list()
    
    for node in leaf_node:
        query_len = len(query)
        node_len = len(node)
        
        if query_len >= node_len: # If query is longer
            for i in range(node_len):
                score = fuzz.partial_ratio(node[i],query)
                if score > threshold:
                    reference_vector.append(node) 
                    
        else: # If leaf node is longer
            for j in range(query_len):
                score = fuzz.partial_ratio(query[j],node)
                if score > threshold:
                    reference_vector.append(node)
    
    reference_vector = list(list(i) for i in set(map(tuple, reference_vector))) # Deduplication
    
    return reference_vector

In [333]:
threshold = 80
query = "Cirrus Parachute First Responders"
reference_vector = fuzzywuzzy_match(query,threshold)
reference_vector_sim = []
for vectors in reference_vector:
    for words in vectors:
        reference_vector_sim.append(words.lower())
        
reference_vector_sim = list(set(reference_vector_sim))

In [334]:
temp = 0
for i in range(19):
    match = set(keyword_vector[i].keys()) & set(reference_vector_sim)
    if len(match) >= 1:
        weight = keyword_vector[i][list(match)[0]]
        if float(weight) >= float(temp):
            temp = weight
            relevant_topic = i

In [335]:
topic_match = [key for key, value in topic_dict.items() if value == relevant_topic]
reply_match_search = forum.iloc[topic_match]

In [336]:
relevant_topic, lda_model.print_topics(num_words = 10)[relevant_topic][1]

(0,
 '0.026*"system" + 0.022*"work" + 0.022*"update" + 0.019*"use" + 0.018*"mfd" + 0.018*"datum" + 0.018*"display" + 0.017*"software" + 0.014*"avidyne" + 0.013*"avionic"')

In [337]:
result = reply_match_search['FormattedBody'].reset_index(drop = True)
for i in range(9):
    print (result[i] + '\n')

If I come over Tuesday do I get to play Igor the lowly apprentice? Since your so conservative, I'll make the one's for the Left sideTrip

In reply to:Usual disclaimers:For display use only!No express or implied warranty of fitness for purpose!May or may not work!Use at you own risk!Do not ingest - for external use only!etc. etc. (don't say you weren't warned!)Fast Eddie:I enjoyed your legal disclaimers.  Last night while watching the LA Clippers play Phoenix I LMAO when I saw the legal disclaimer at the bottom of this ad Click Here.I hope Dennis can give us some legal expertise on the disclaimer in this ad.[;)]

Set going out today to Mike Danko.BTW, the reason I post this here is:1) to bump the thread to the top for newbies, and,2) to keep a record of who I've sent them to and whenEnjoy!

Fast Eddie,I made two of these up when you first introduced them. That was quite a while ago. Home here in Vermont, I have the plane hangered, But when I go to Florida I use them. I also use them whe

__Save the model__

In [90]:
# Save model to disk.
temp_file = datapath("lda.model")
lda_model.save(temp_file)

# Load lda.model
lda = gensim.models.ldamodel.LdaModel.load(temp_file)

In [22]:
# pd.DataFrame(lda_model.print_topics(num_words = 10), columns = ['Topic', 'Keywords']).to_csv('LDA_vector.csv')

In [11]:
df.head()

,Date,Title,Content,Author,Like,Comment,Resource,SourceScore,RecencyRate,AuthorScore
0,2015-01-01,President's Column,JANUARY FEBRUARY 20154CIRRUS PILOTAs this is b...,NaN,0,0,Magazine,3,-0.465959,-1.209951
1,2015-01-01,COPA News,JANUARY FEBRUARY 20156CIRRUS PILOTWith this is...,NaN,0,0,Magazine,3,-0.465959,-1.209951
2,2015-01-01,Regional News,JANUARY FEBRUARY 201512CIRRUS PILOTby GIL WILL...,GIL WILLIAMSON,0,0,Magazine,3,-0.465959,0.211050
3,2015-01-01,Cirrus Perspective,JANUARY FEBRUARY 201518CIRRUS PILOTIts hard to...,NaN,0,0,Magazine,3,-0.465959,-1.209951
4,2015-01-01,Member Spotlight,JANUARY FEBRUARY 201522CIRRUS PILOTCirrus Pilo...,KIM BLONIGEN,0,0,Magazine,3,-0.465959,0.211050


__Input Searching query__

For Search Results Ranking

- "Cirrus Parachute First Responders"
- "Different between AATD and FTD"
- "Activate vector to final"
- "Turbo normalize vs turbo charge"
- "Shooting ILS approach"

In [19]:
def extract_keywords(search_query):
    '''
    Get the most important keyword from searching query 
    '''
    stop_words = stopwords.words('english')
    stop_words.append('vs')
    r = Rake(stopwords = stop_words)
    
    r.extract_keywords_from_text(search_query)
    rank = r.get_ranked_phrases_with_scores()
    itemMaxValue = max(rank, key=lambda x: x[0])[0]
    
    # Find the most important keyword
    listOfKeys = list()
    for value, key in rank:
        if value == itemMaxValue:
            listOfKeys.append(key.lower())
    
    return listOfKeys

In [20]:
# search_query = ["Cirrus Parachute First Responders", "Difference between AATD and FTD", "Activate vector to final", "Turbo normalize vs turbo charge", "Shooting ILS approach"]
# extract_keywords("Difference between AATD and FTD")

In [118]:
taxonomy_terms = [tree['Title'] for tree in taxonomy_data['data'][1:]]

topic_vector = {}
for i in range(len(taxonomy_terms)):
    topic_info = taxonomy_data['data'][1:][i] # Sub level of all terms other than COPA
    title = re.findall(r'Title\': \'(.*?)\'', str(topic_info)) # Extract all children titles under specific term
    if len(title) > 1: # If the term is among the bottom level, then just assign itself to the term
        topic_vector[taxonomy_terms[i].lower()] = title[1:]
    else:
        topic_vector[taxonomy_terms[i].lower()] = [taxonomy_terms[i]]

Think about which fuzzywuzzy method we should use:
For each vector in topic_vector: which level we should match to.
What kind of vector we want to get in the end? 
Current version: Combine all the match vectors together.


In [119]:
threshold = 80  # Maybe adjusted to improve performance

In [125]:
# Compare query to each leaf node, if the partial match score is higher than the threshold, then return a combination vector
# that contains all words in those leaf nodes.
def fuzzywuzzy_match(query,threshold):
    
    # Generate a list contains all leaf nodes, each sublist contains all words in that leaf node.
    leaf_node = list()
    for item in topic_vector:
        if len(topic_vector[item]) == 1:
            leaf_node.append(topic_vector[item][0].split())
            
    reference_vector = list()
    
    for node in leaf_node:
        query_len = len(query)
        node_len = len(node)
        
        if query_len >= node_len: # If query is longer
            for i in range(node_len):
                score = fuzz.partial_ratio(node[i],query)
                if score > threshold:
                    reference_vector.append(node) 
                    
        else: # If leaf node is longer
            for j in range(query_len):
                score = fuzz.partial_ratio(query[j],node)
                if score > threshold:
                    reference_vector.append(node)
    
    reference_vector = list(list(i) for i in set(map(tuple, reference_vector))) # Deduplication
    
    return reference_vector

In [183]:
query = "Cirrus Parachute First Responders"
fuzzywuzzy_match(query,threshold)

[['Parachute'],
 ['Cirrus', 'Airframe', 'Parachute', 'System'],
 ['Cirrus', 'Approach'],
 ['Parachute', 'Disreef'],
 ['Over', 'a', 'Runaway'],
 ['Reefed', 'Parachute'],
 ['Rocket', 'and', 'Parachute', 'Extraction'],
 ['Not', 'Over', 'a', 'Runaway']]

In [97]:
def exact_match(search_query):
    '''
    The searching query can be exact matched by taxonomy tree.
    '''
    if search_query in taxonomy_terms:
        taxonomy_vector = topic_vector[search_query.lower()]
    return taxonomy_vector

In [101]:
# exact_match('Tire Brands')

['Goodyear', 'Michelin', 'Aero Classic', 'STA']

In [99]:
# Create a vector that stores all the phrases(more than one word) that should be 
# token as a whole.
def generate_phrases_tokenizer(topic_vector): # Return a list of tuples containing phrases.
    all_multiword_tokens = list()
    for item in topic_vector: 
        for phrase in topic_vector[item]:
            phrase = phrase.replace('(','') # Eliminate '(' and ')' in the phrase
            phrase = phrase.replace(')','')
           #phrase = re.sub(r'( \(.*\))','',phrase) #exclude all contents in '()'
            phrase_lower = phrase.lower() # Convert all characters into lower case.
            #phrase_lower.replace('-', ' ')
            word_lst = phrase_lower.split()
            if len(word_lst) >= 2: 
                word_tuple = tuple(word_lst)
                all_multiword_tokens.append(word_tuple)  
    all_multiword_tokens = list(set(all_multiword_tokens)) # Deduplication
    tokenizer = MWETokenizer(all_multiword_tokens,separator=' ')
    return tokenizer

In [100]:
def any_match(search_query,tokenizer):
    '''
    Any words in searching query can be exact matched by taxonomy tree.
    '''
    # Split input string into words or phrases according to taxonomy tree.
    words = tokenizer.tokenize(search_query.split())
    taxonomy_vector = []
    for word in words:
        try:
            taxonomy_vector.append(topic_vector[word])
        except:
            continue
    return taxonomy_vector

In [84]:
phrase_tokenizer = generate_phrases_tokenizer(topic_vector)
any_match("Turbo normalize vs turbo charge", phrase_tokenizer)

[['Turbo']]

In [191]:
def keyword_match(keywords):
    '''
    The most important keyword of searching query can be matched by taxonomy tree.
    '''
    taxonomy_vector = []
    for keyword in keywords:
        try:
            taxonomy_vector.append(topic_vector[keyword])
        except:
            continue
    return taxonomy_vector

In [229]:
# keywords = extract_keywords("Engine doesn't work.")
# keyword_match(keywords), keywords

([['Engine']], ['work', 'engine'])

In [95]:
def topic_match(search_query):
    '''
    Extract the topic of searching query by LDA model trained by forum posts.
    '''
    texts1 = [[word for word in doc.lower().split() if word not in stop_words] for doc in [search_query]]
    corpus1 = [id2word.doc2bow(t) for t in texts1]
    result = lda_model.get_document_topics(corpus1)
    LDA_topic = max(result[0], key=lambda x: x[1])[0]
    return LDA_topic

In [82]:
# topic_match('Turbo normalize vs turbo charge'), lda_model.print_topics(num_words = 10)[12]

(12,
 (12,
  '0.101*"altitude" + 0.083*"high" + 0.065*"climb" + 0.041*"low" + 0.034*"performance" + 0.032*"gun" + 0.030*"turbo" + 0.028*"level" + 0.028*"foot" + 0.027*"rate"'))